# Pytorch with Examples

本教程通过自包含的示例介绍 PyTorch 的基本概念。PyTorch 的核心有两个主要功能：

+ n 维张量，类似于 numpy，但可以在 GPU 上运行；
+ 用于构建和训练神经网络的自动微分

我们将使用一个三阶多项式拟合 $y=sin(x)$ 的问题作为运行示例。网络将有四个参数，并将使用梯度下降法进行训练，通过最小化网络输出与真实输出之间的欧氏距离来拟合随机数据。

> 您可以浏览本页末尾的[各个示例](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#examples-download)。

## Tensor

### numpy

在介绍 PyTorch 之前，我们先用 numpy 实现网络。Numpy 提供了一个 n 维数组对象，以及许多用于操作这些数组的函数。Numpy 是一个用于科学计算的通用框架；它对计算图、深度学习或梯度一无所知。不过，通过使用 numpy 操作手动实现网络的前向和后向传递，我们可以轻松地使用 numpy 将三阶多项式拟合为正弦函数：

In [4]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# 创建随机的输入和输出数据
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# 随机初始化权重
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # 前向计算：计算预测 y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 计算和输出损失
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)
    
    # 反向计算：计算 a,b,c,d 相对于 loss 的梯度值
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f"Result: y = {a} + {b}x + {c}x^2 + {d}x^3")



99 1962.3593958622448
199 1306.4983338251932
299 871.0853692527345
399 581.9475262483453
499 389.89107737722884
599 262.2823083159039
699 177.46832437379572
799 121.07888866003591
899 83.57470032693371
999 58.621726760761725
1099 42.01306985309979
1199 30.953813294272425
1299 23.586545005790533
1399 18.676488761886517
1499 15.402504851043869
1599 13.218328367534383
1699 11.760416116126226
1799 10.786729501989374
1899 10.136055955748075
1999 9.700969573169461
Result: y = -0.014130940290577368 + 0.8309195784189938x + 0.0024378223399859947x^2 + -0.08965753899700578x^3


### Pytorch: Tensors

numpy 是一个很棒的框架，但它无法利用 GPU 来加速其数值计算。对于现代深度神经网络，GPU 通常可以提供 50 倍或更高的加速，因此不幸的是，numpy 不足以满足现代深度学习的需求。

在这里，我们介绍最基本的 PyTorch 概念：Tensor。PyTorch 张量在概念上与 numpy 数组相同：张量是一个 n 维数组，PyTorch 提供了许多用于操作这些张量的函数。在幕后，Tensor 可以跟踪计算图和梯度，但它们也可用作科学计算的通用工具。

与 numpy 不同的是，PyTorch 张量可以利用 GPU 来加速其数值计算。要在 GPU 上运行 PyTorch 张量，您只需指定正确的设备即可。

在这里，我们使用 PyTorch 张量将三阶多项式拟合为正弦函数。与上面的 numpy 示例类似，我们需要手动实现通过网络的前向和后向传递：

In [5]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # 注释该行以运行在 GPU 上

# 创建随机输入和输出
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 随机初始化权重
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # 前向计算
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 计算和输出损失
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # 反向
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 使用梯度下降更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f"Result: y = {a.item()} + {b.item()}x + {c.item()}x^2 + {d.item()}x^3")

99 1721.864501953125
199 1185.115234375
299 817.46044921875
399 565.3399658203125
499 392.25030517578125
599 273.28314208984375
699 191.42315673828125
799 135.0342559814453
899 96.14817810058594
999 69.30343627929688
1099 50.751808166503906
1199 37.91786193847656
1299 29.030498504638672
1399 22.87004280090332
1499 18.59565544128418
1599 15.627120018005371
1699 13.563624382019043
1799 12.127979278564453
1899 11.128321647644043
1999 10.43166732788086
Result: y = 0.03838726878166199 + 0.8735573887825012x + -0.006622442975640297x^2 + -0.09572239220142365x^3


## 自动微分 - Autograd

### Pytorch: Tensor 和 autograd

在上述示例中，我们不得不手动实现神经网络的前向和反向传递。手动实现反向传递对于小型双层网络来说不算什么，但对于大型复杂网络来说，很快就会变得非常棘手。

值得庆幸的是，我们可以使用[自动微分](https://en.wikipedia.org/wiki/Automatic_differentiation)来自动计算神经网络的反向传递。PyTorch 中的 `autograd` 软件包正是提供了这种功能。使用 `autograd` 时，网络的前向传递将定义一个**计算图**；图中的**节点**将是张量，**边**将是通过输入张量产生输出张量的函数。通过该图的**反向传播**，可以轻松计算梯度。

这听起来很复杂，但实际使用起来却非常简单。每个张量代表计算图中的一个节点。如果 `x` 是一个 `x.requires_grad=True` 的张量，那么 `x.grad` 就是另一个张量，包含 `x` 相对于某个标量值的梯度。在这里，我们使用 PyTorch 张量和 `autograd` 来实现正弦与三阶多项式拟合的示例；现在我们不再需要手动实现网络的后向传递：

In [4]:
import torch
import math

# 让模型在加速器上进行训练，例如：CUDA, MPS, MTIA
# 如果当前的加速器可以使用，那么就使用它；否则，使用 CPU
dtype = torch.float
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
torch.set_default_device(device)

# 创建张量保存输入和输出
# 默认情况下，requires_grad=False，也就是表明在反向传播过程中，
# 我们不需要计算这些张量对应的梯度
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

# 创建权重的随机值
# 对于三阶多项式，我们需要四个权重：y = a + b x + c x^2 + d x^3
# 设置 requires_grad=True 以为只我们想要在反向传播过程中，
# 计算相应张量的梯度值
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # 前向计算：在输入上使用操作计算预测值 y
    y_pred = a + b * x + c * x**2 + d * x**3

    # 计算并输出损失值
    # 损失值为一个形状为 (1,) 的张量
    # loss.item() 获取损失的标量值
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # 使用自动梯度计算反向传播过程
    # 该调用会计算损失值相对于所有 requires_grad=True 的张量
    # 该操作以后，a.grad, b.grad, c.grad, d.grad 为分别保存着
    # 损失值相对于 a,b,c,d 的梯度张量值
    loss.backward()

    # 手动使用梯度下降更新权重，之所以用 torch.no_grad() 进行包围
    # 因为 requires_grad=True 的权重，但是在自动微分过程中，我们
    # 不需要跟踪梯度值
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 在更新完权重以后，把所有梯度值手动设置为 0
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None


print(f"Result: y = {a.item()} + {b.item()}x + {c.item()}x^2 + {d.item()}x^3")

[ROCT-Thunk-Interface/src/hymgr.c:309, WARNING]: Version mismatch


Using cuda device
99 1412.75537109375
199 943.9651489257812
299 631.9915771484375
399 424.2935791015625
499 285.95855712890625
599 193.78067016601562
699 132.3299560546875
799 91.34317779541016
899 63.991355895996094
999 45.728450775146484
1099 33.527069091796875
1199 25.37053680419922
1299 19.91449546813965
1399 16.262409210205078
1499 13.816181182861328
1599 12.17640209197998
1699 11.076427459716797
1799 10.337980270385742
1899 9.841805458068848
1999 9.508166313171387
Result: y = -0.01516183651983738 + 0.8353146910667419x + 0.002615669509395957x^2 + -0.09028270840644836x^3


### Pytorch: 定义新的自动梯度函数

在底层，每个原始自动求导运算符实际上是两个对张量进行操作的函数。前向函数根据输入张量计算输出张量。后向函数接收输出张量相对于某个标量值的梯度，并计算输入张量相对于同一标量值的梯度。

在 PyTorch 中，我们可以通过定义 `torch.autograd.Function` 的子类并实现前向和反向函数来轻松定义我们自己的自动求导运算符。然后，我们可以通过构造一个实例并像函数一样调用它来使用我们的新自动求导运算符，传递包含输入数据的张量。

在这个例子中，我们定义我们的模型为 $y=a+bP_3(c+dx)$，而不是$y=a+bx+cx^2+dx^3$，其中$P_3(x)=\frac{1}{2}(5x^3-3x)$是[三次勒让德多项式](https://en.wikipedia.org/wiki/Legendre_polynomials)，我们编写我们自定义的自动微分函数，以计算$P_3$的前向和反向，并使用它来实现我们的模型：

In [2]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    """
    通过继承 torch.autograd.Function 我们可以实现自定义自动微分算子
    同时在输入张量上实现前向和反向过程
    """
    @staticmethod
    def forward(ctx, input):
        """
        在前向过程中，我们会获取到一个包含输入的张量，并返回一个包含输出
        的张量。ctx 是一个上下文对象，可用于存储用于反向计算的信息。
        您可以使用 ctx.save_for_backward 方法缓存任意对象以供反向传递使用。
        """
        ctx.save_for_backward(input)
        return 0.5*(5 * input**3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播过程中，我们会获取到一个包含损失相对于输出的梯度张量，
        并且需要我们计算损失相对于输入的梯度值。
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)
    
dtype = torch.float
# device = torch.device("cpu")
device = torch.device("cuda:0")

# 创建保存输入和输出的张量
# 默认情况下，requires_grad=False，也就是意味着在反向传播过程中，
# 我们不需要计算相对于这些张量的梯度值
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 创建权重的随机张量，在该例子中，我们需要四个权重：y = a + b * P3(c + d * x)
# 这些权重需要初始化得离正确结果不太远，以确保收敛。
# 设置requires_grad=True 表示我们想要在反向传播过程中计算这些张量的梯度。
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # 为了应用我们的算子，我们使用 Function.apply 方法
    # 给方法一个匿名名称：P3
    P3 = LegendrePolynomial3.apply

    # 前向过程：使用前向算子计算预测值 y
    # 使用自定义自动微分操作计算 P3
    y_pred = a + b * P3(c + d * x)

    # 计算并输出损失
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # 使用自动微分计算反向
    loss.backward()

    # 更新权重
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad
    
        # 在更新权重以后，手动设置梯度为 0
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.9583282470703
199 144.66018676757812
299 100.70249938964844
399 71.03518676757812
499 50.97853088378906
599 37.40312957763672
699 28.206863403320312
799 21.973182678222656
899 17.745725631713867
999 14.877887725830078
1099 12.931764602661133
1199 11.610918045043945
1299 10.714256286621094
1399 10.10548210144043
1499 9.69210433959961
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = 4.1149544882657096e-10 + -2.208526849746704 * P3(-5.391238566687662e-10 + 0.2554861009120941 x)


## `nn` 模块

### Pytorch: `nn`

计算图和自动求导是定义复杂运算符和自动求导的非常强大的范例；然而，对于大型神经网络来说，原始的自动求导可能有点太低级了。

在构建神经网络时，我们经常考虑将计算安排到层中，其中一些层具有可学习的参数，这些参数将在学习过程中进行优化。

在 TensorFlow 中，[Keras](https://github.com/fchollet/keras)、[TensorFlow-Slim](https://github.com/google-research/tf-slim) 和 [TFLearn](http://tflearn.org/) 等软件包提供了对原始计算图的更高级别的抽象，这对于构建神经网络非常有用。

在 PyTorch 中，`nn` 包也用于同样的目的。`nn` 包定义了一组模块，它们大致相当于神经网络层。模块接收输入张量并计算输出张量，但也可能保存内部状态，例如包含可学习参数的张量。`nn` 包还定义了一组有用的损失函数，这些函数在训练神经网络时很常用。

在这个例子中，我们使用 `nn` 包来实现我们的多项式模型网络：

In [3]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        在构造函数中，我们实例化四个参数，并将他们赋值为成员参数
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        在前向算子中，我们接受一个输入张量，并必须返回一个输出张量。
        我们可以使用构造函数中定义的模块以及张量上的任意运算符。
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
    
    def string(self):
        """
        就像 Python 中的任何类一样，你也可以在 PyTorch 模块上定义自定义方法
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'
    
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 通过实例化类，构造我们的模型
model = Polynomial3()

# 构建我们的损失函数和优化器。SGD 构造函数中对 model.parameters() 的调用
# 将包含可学习参数（使用 torch.nn.Parameter 定义）它们是模型的成员。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

for t in range(2000):
    # 前向
    y_pred = model(x)

    # 计算损失并输出
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 设置梯度为 0，执行反向传递，并更新权重。
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 2819.638916015625
199 1924.453857421875
299 1315.8544921875
399 901.6302490234375
499 619.3822021484375
599 426.8427429199219
699 295.3485107421875
799 205.44180297851562
899 143.8990020751953
999 101.72352600097656
1099 72.78728485107422
1199 52.91194152832031
1299 39.24473571777344
1399 29.836050033569336
1499 23.35184097290039
1599 18.878271102905273
1699 15.788518905639648
1799 13.652347564697266
1899 12.1738920211792
1999 11.149667739868164
Result: y = 0.043910812586545944 + 0.8327017426490784 x + -0.007575344294309616 x^2 + -0.08991103619337082 x^3


### Pytorch：控制流和权重共享

作为动态计算图和权重共享的一个例子，我们实现了一个非常奇怪的模型：一个三阶五阶多项式，在每次前向传递时选择一个 3 到 5 之间的随机数并使用那么多阶，多次重复使用相同的权重来计算四阶和五阶。

对于这个模型，我们可以使用普通的 Python 流控制来实现循环，我们可以通过在定义前向传递时多次重复使用相同的参数来实现权重共享。

我们可以轻松地将此模型实现为 `Module` 子类：

In [4]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        在构造函数中，我们实例化五个参数，并将其作为成员
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        对于模型的前向传递，我们随机选择 4 或 5

        并重用 e 参数来计算这些顺序的贡献。

        由于每个前向传递都会构建一个动态计算图，因此在定义模型的前向传递时，我们可以使用正常的
        Python 控制流运算符，如循环或条件语句。

        在这里我们还看到，在定义计算图时多次重用相同的参数是完全安全的。
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        就像 Python 中的任何类一样，你也可以在 PyTorch 模块上定义自定义方法
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 通过实例化上面定义的类来构建我们的模型
model = DynamicNet()

# 构建我们的损失函数和优化器。
# 使用原始随机梯度下降法训练这个奇怪的模型很困难，所以我们使用动量
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    y_pred = model(x)

    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 1063.7928466796875
3999 475.17193603515625
5999 215.9987030029297
7999 96.30238342285156
9999 48.99087905883789
11999 26.618961334228516
13999 16.54855728149414
15999 12.227919578552246
17999 10.307435035705566
19999 9.391677856445312
21999 9.107739448547363
23999 8.944269180297852
25999 8.894832611083984
27999 8.864897727966309
29999 8.853294372558594
Result: y = 0.00012930684897582978 + 0.8543539643287659 x + -0.0006214152672328055 x^2 + -0.09329459816217422 x^3 + 0.00011243280459893867 x^4 ? + 0.00011243280459893867 x^5 ?
